### Семинар 4. Классы. Продолжение.

<br />

##### Виртуальные функции

Упражение: какая функция будет вызвана. Нарисовать что будет происходить в памяти

```c++
class A
{
public:
	~A() {}

	void f() {}
	virtual void g() {}
	void h() {}
};

class B : public A
{
public:
	virtual ~B() {}

	virtual void f() {}
	void g() {}
	void h() {}
};

class C : public B
{
public:
	~C() {}

	void f() {}
	void g() {}
	virtual void h() {}
};

int main()
{
	{
		C c;
		c.f(); c.g(); C.h();

		B b = c;
		b.f(); b.g(); b.h();

		A& ar = c;
		ar.f(); ar.g(); ar.h();
	}

	{
		C* cp = new C;
		A* ap = cp;
		ap->f(); ap->g(); ap->h();
		delete ap;
	}

	{
		C* cp = new C;
		B* bp = cp;
		delete bp;
	}

	return 0;
}
```

<br />

Упражение: какая функция будет вызвана.

```c++
class A {
public:
    A() {
        init();
    }

    ~A() {
        deinit();
    }

    virtual void init();
    virtual void deinit();
};

class B : public A {
public:
    B() {}
    ~B() {}

    void init() override { ... }
    void deinit() override { ... }
};

int main() {
    B b;
    return 0;
}
```

<br />

##### Виртуальные деструкторы: зачем и как

Мы уже познакомились с механизмом виртуальных функций и поняли зачем и как они вызываются. Насчёт деструкторов рассмотрим такой пример:

```c++
class Animal
{
    int age;
    
public:
    virtual void cry() = 0;
};

class Dog : public Animal
{
    std::string name;
    
public:
    void cry() override;
};

class Human : public Animal
{
    std::string name;
    std::string surname;
    
public:
    void cry() override;
};


void func()
{
    Animal* animal = nullptr;
    if (std::rand() % 2)
        animal = new Dog;
    else
        animal = new Human;    
    ...;    
    delete animal;  // что здесь делать компилятору?
}
```



Поэтому реализуем иерархии так:

```c++
class Animal
{
    int age;

public:
    virtual ~Animal() = default;  // такой деструктор - ВИРТУАЛЬНЫЙ метод
    virtual void cry() = 0;
};

// далее как обычно
```

<br />

##### const-методы

Упражнение: какая функция будет вызвана

```c++
class A
{
public:
	void f() { std::cout << "f();"; }
	void f() const { std::cout << "f() const;" }
};


int main()
{
	A a;
	a.f();

	const A ca;
	ca.f();

	const A& car = a;
	car.f();

	A& ar = a;
	ar.f();

	A* const acp = &a;
	acp->f();

	const A* cap = &a;
	cap->f();

	const A a2 = a;
	a2.f();

	return 0;
}
```

<br />

##### class vs struct

* `class` - если есть инвариант
* `struct` - если поля независимы

```c++
struct Point
{
	double x;
	double y;
};

struct Color
{
	char r;
	char g;
	char b;
};

// ???
struct Size
{
	double w;
	double h;
};

struct Rectangle
{
	Point origin;
	Size size;
};

class JuiceBottle
{
private:
	double max_volume_;
	double cur_volume_;
};

class String
{
private:
	char *s_;
	size_t len_;
	size_t capacity_;
};
```

<br />

##### Принципы дизайна класса

* RAII
* Есть инвариант - class, поля независимы - struct
* Если есть подклассы - делаем virtual деструктор
* Правило 6
    * =default где достаточно автогенерённого
    * =delete где нужно запретить
    
    ```c++
    class Animal
    {
    private:
        std::string name;

    public:
        Animal() : name("unknown") {}
        Animal(const Animal&) = default;
        Animal(Animal&&) noexcept = delete;
        Animal& operator= (const Animal&) = default;
        Animal& operator= (Animal&&) = delete;
        ~Animal() noexcept = default;
    };
    ```

* noexcept d-tor, move operators
* noexcept на всё что не кидает исключений
* const на все методы, которые не меняют состояния класса (значения полей)

    ```c++
    class Animal
    {
    private:
        std::string name;

    public:
        const std::string& get_name() const noexcept { return name; }
    };
    ```

* override на переопределённые virtual методы

    ```c++
    class Animal
    {
    public:
        virtual void say() = 0;
    };

    class Cat : public Animal
    {
    public:
        void say() override;
    };
    ```

* Какой функционал делать методом, а какой - свободной фнуцией? Всопним ADL. Благодаря ему есть философский принцип, что функции, принимающие первым аргументом объект класса (или ссылку), являются частью интерфейса класса:

    ```c++
    struct Vector3D
    {
        double x, y, z;
    };

    // философски часть интерфейса класса Vector3D
    double length(const Vector3D v);
    ```

    Поэтому принцип рационального минмимума в методах:
      * struct Vector3D - всё в свободные функции (normalize, length, rotate ...))
      * vector<int> (pointer, size, capacity: size() - member; at() - member; sort() - function; empty() - ???; middle element - function; max() - function, mean() - function)
      * cache (add, get, del; clear? - в метод, т.к. эффективнее реализовать, обладая доступом до полей)

    Нужна __причина__, чтобы что-нибудь сделать методом:
        * требуется доступ до приватных полей и нельзя реализовать иначе (vector.size(), vector.at(), vector.push_back())
        * реализация через приватные поля проще и быстрее (cache.clear())
        * лучше вписывается в сщуествующие методы класса (vector.empty())
        * что-нибудь ещё?

* Никогда не используйте protected-поля

* Иногда нужно менять поля в const-методах, и это не влияет на состояние класса в умозрительном смысле. Используйте mutable:
    * Кеширование: бинарное дерево, которое запоминает последний запрос
    * mutex
    * отложенная инициализация

* SRP - Single Responsibility Principle - Принцип единственной ответственности - класс должен быть экспертом в одной и только одной области. Такие классы проще отлаживать и тестировать. Всегда понятно, ошибка в рамках его области ответственности или нет.
* Liskov Substitution Principle - Принцип подстановки Барбары Лисков - наследник класса всегда можно передать в алгоритм вместо базового, при этом поведение алгоритма не изменится. Другими словами: наследники расширяют поведение базовых классов, но не изменяют его.